# Titanic problem

## Use XGBoost

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix

In [2]:
train_df = pd.read_csv('train.csv')

In [3]:
def get_surname(item):
    return item['Name'].split(',')[0]

def get_n_members(item):
    return (item['SibSp']+item['Parch'] +
           1) # self

def get_passengers_ticket_numbers(passengers):
    tickets = {p['Ticket'] for p in passengers}
    if len(tickets) <= 1:
        return tickets.pop()
    else:
        return tickets

# Holders of close ticket numbers are likely to be family members.
def family_like(item, fam_members):
    ticket = item['Ticket']
    for member in fam_members:
        mem_ticket = member['Ticket']
        if ticket == mem_ticket:
            return True
        if ticket.isdigit() and mem_ticket.isdigit() and abs(int(ticket) - int(mem_ticket)) <= 2:
            return True
    return False

def group_families(df):
    families = {}

    for i, (column_name, item) in enumerate(df.iterrows()):
        fam_name = get_surname(item)
        ticket_num = item['Ticket']
        fam_dict = families.setdefault(fam_name, {})
        for cnt in range(100):
            fam_name_mod = f'{fam_name}#{cnt}'
            if fam_name_mod in fam_dict:
                if family_like(item, fam_dict[fam_name_mod]):
                    fam_dict[fam_name_mod].append(item)
                    break
            else:
                fam_dict.setdefault(fam_name_mod, []).append(item)
                break
    return families

def split_single_family_passengers(df, families):
    single_indices = set()
    family_indices = set()
    for fam_name, subfamilies in families.items():
        for _, passengers in subfamilies.items():
            if len(passengers) <= 1:
                single_indices.add(passengers[0]['PassengerId'])
            else:
                for p in passengers:
                    family_indices.add(p['PassengerId'])
    return df[df['PassengerId'].isin(sorted(single_indices))], df[df['PassengerId'].isin(sorted(family_indices))]

families = group_families(train_df)
single_df, family_df = split_single_family_passengers(train_df, families)

## Prepare data

In [4]:
def prepare_data(df):
    # This makes df['Sex'] viewing instead of copying...?
    df = pd.DataFrame(df)
    df = df.dropna(subset=['Age'])
    df['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
    df = pd.get_dummies(df, columns=['Embarked'])

    X_df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]
    y_df = df[['Survived']]
    X_keys = list(X_df.keys())
    X_values = X_df.values
    y_values = np.squeeze(y_df.values)

    X_train, X_test, y_train, y_test = train_test_split(
        X_values, y_values, test_size=0.3, random_state=1, stratify=y_values)
    return X_train, X_test, y_train, y_test, X_keys

## Use XGBoost

In [5]:
def prob2index(prob):
    return np.array([np.argmax(ps) for ps in prob])

def train_model(X_train, y_train):
    params = {
        'objective':'multi:softprob',
        'eval_metric': 'mlogloss',
        'num_class': 2,
        'max_depth': 3,
        'eta': 0.1,
        'verbosity': 0,
        'random_state': 71,
    }
    
    accs = []
    models = []

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

    for fold_id, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
        X_tr  = X_train[train_idx, :]
        X_val = X_train[val_idx, :]
        y_tr  = y_train[train_idx]
        y_val = y_train[val_idx]
        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dvalid = xgb.DMatrix(X_val, label=y_val)
        watchlist = {(dtrain, 'train'), (dvalid, 'eval')}

        bst = xgb.train(params, dtrain, num_boost_round=10, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)
        y_pred = prob2index(bst.predict(dvalid))
        conf_mat = confusion_matrix(y_val, y_pred)
        val_acc = np.trace(conf_mat) / len(y_val)
        accs.append(val_acc)
        models.append(bst)
        print(f'[{fold_id}] val acc={val_acc}')

    best_idx = np.argmax(accs)
    best_model = models[best_idx]
    print(f'{best_idx=}')
    return best_model

## Train and evaluation for single

In [6]:
X_train_s, X_test_s, y_train_s, y_test_s, _ = prepare_data(single_df)
model = train_model(X_train_s, y_train_s)
dtest = xgb.DMatrix(X_test_s)
y_pred = prob2index(model.predict(dtest))
conf_mat = confusion_matrix(y_test_s, y_pred)
print(conf_mat)
print('acc:', np.trace(conf_mat) / len(y_test_s))

[0] val acc=0.8235294117647058
[1] val acc=0.7761194029850746
[2] val acc=0.7313432835820896
[3] val acc=0.8507462686567164
[4] val acc=0.8805970149253731
best_idx=4
[[82  8]
 [19 36]]
acc: 0.8137931034482758


## Train and evaluation for family

In [7]:
X_train_f, X_test_f, y_train_f, y_test_f, _ = prepare_data(family_df)
model = train_model(X_train_f, y_train_f)
dtest = xgb.DMatrix(X_test_f)
y_pred = prob2index(model.predict(dtest))
conf_mat = confusion_matrix(y_test_f, y_pred)
print(conf_mat)
print('acc:', np.trace(conf_mat) / len(y_test_f))

[0] val acc=0.8484848484848485
[1] val acc=0.8484848484848485
[2] val acc=0.8181818181818182
[3] val acc=0.84375
[4] val acc=0.84375
best_idx=0
[[35  2]
 [ 9 24]]
acc: 0.8428571428571429
